In [ ]:
!pip install roboflow
!pip install ultralytics==8.3.40

import os
import cv2
import shutil
import matplotlib.pyplot as plt

from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import drive
drive.mount('/content/gdrive')

ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

In [ ]:
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("smart-fridge-p5rw0")
version = project.version(9)
dataset = version.download("yolov11")

# Validation

In [ ]:
model = YOLO(os.path.join(ROOT_DIR, "Level2_train/detect/train/weights/best.pt"))

metrics = model.val(data="/content/Smart-Fridge-9/data.yaml", split="test")

class_names = model.names

for i, name in class_names.items():
    precision, recall, ap50, ap = metrics.box.class_result(i)
    f1 = (2 * precision * recall) / (precision + recall + 1e-6)
    print(f"Class: {name}")
    print(f" - Precision: {precision:.4f}")
    print(f" - Recall: {recall:.4f}")
    print(f" - F1 Score: {f1:.4f}")
    print(f" - AP@0.5: {ap50:.4f}")
    print(f" - AP@0.5:0.95: {ap:.4f}")
    print("-" * 30)

In [ ]:
src = '/content/runs'
dst = '/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level2_val'

shutil.copytree(src, dst)

# Prediction

In [ ]:
model = YOLO(os.path.join(ROOT_DIR, "Level2_train/detect/train/weights/best.pt"))
image_paths = ["/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation/sim16.jpg"]

processed_images = []
for image_path in image_paths:
    img = cv2.imread(image_path)
    processed_images.append(img)

results = model(processed_images)

for result in results:
    boxes = result.boxes
    masks = result.masks
    keypoints = result.keypoints
    probs = result.probs
    obb = result.obb

    predicted_classes = [model.names[int(cls)] for cls in result.boxes.cls]
    print("Predicted classes:", predicted_classes)

    result.show()
    result.save(filename="result.jpg")